<a href="https://colab.research.google.com/github/Salah0Uddin/NLP_Learning/blob/main/NLP_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fake News Classifier Using LSTM

In [1]:
import pandas as pd

In [2]:
import pandas as pd

df = pd.read_csv("/content/train (2).csv", sep=';', encoding='utf-8-sig', on_bad_lines='skip')

# Optional: clean column names
df.columns = df.columns.str.strip()  # removes spaces or stray characters

# Check column names
print(df.columns)


Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')


In [3]:
df

,Unnamed: 0,title,text,label
0,0,Palestinians switch off Christmas lights in Be...,"RAMALLAH, West Bank (Reuters) - Palestinians s...",1
1,1,China says Trump call with Taiwan president wo...,BEIJING (Reuters) - U.S. President-elect Donal...,1
2,2,FAIL! The Trump Organization’s Credit Score W...,While the controversy over Trump s personal ta...,0
3,3,Zimbabwe military chief's China trip was norma...,BEIJING (Reuters) - A trip to Beijing last wee...,1
4,4,THE MOST UNCOURAGEOUS PRESIDENT EVER Receives ...,There has never been a more UNCOURAGEOUS perso...,0
...,...,...,...,...
24348,24348,Mexico Senate committee OK's air transport dea...,MEXICO CITY (Reuters) - A key committee in Mex...,1
24349,24349,BREAKING: HILLARY CLINTON’S STATE DEPARTMENT G...,IF SHE S NOT TOAST NOW THEN WE RE IN BIGGER TR...,0
24350,24350,trump breaks from stump speech to admire beaut...,kremlin nato was created for agression \nruss...,0
24351,24351,NFL PLAYER Delivers Courageous Message: Stop B...,Dallas Cowboys star wide receiver Dez Bryant t...,0


In [4]:
df.rename(columns={'Unnamed: 0': 'id'}, inplace=True)


In [5]:
df

,id,title,text,label
0,0,Palestinians switch off Christmas lights in Be...,"RAMALLAH, West Bank (Reuters) - Palestinians s...",1
1,1,China says Trump call with Taiwan president wo...,BEIJING (Reuters) - U.S. President-elect Donal...,1
2,2,FAIL! The Trump Organization’s Credit Score W...,While the controversy over Trump s personal ta...,0
3,3,Zimbabwe military chief's China trip was norma...,BEIJING (Reuters) - A trip to Beijing last wee...,1
4,4,THE MOST UNCOURAGEOUS PRESIDENT EVER Receives ...,There has never been a more UNCOURAGEOUS perso...,0
...,...,...,...,...
24348,24348,Mexico Senate committee OK's air transport dea...,MEXICO CITY (Reuters) - A key committee in Mex...,1
24349,24349,BREAKING: HILLARY CLINTON’S STATE DEPARTMENT G...,IF SHE S NOT TOAST NOW THEN WE RE IN BIGGER TR...,0
24350,24350,trump breaks from stump speech to admire beaut...,kremlin nato was created for agression \nruss...,0
24351,24351,NFL PLAYER Delivers Courageous Message: Stop B...,Dallas Cowboys star wide receiver Dez Bryant t...,0


In [6]:
df.isna().sum()

,0
id,0
title,0
text,0
label,0


In [7]:
df=df.dropna()

In [8]:
#seperating independent and dependent labels
X=df.drop('label',axis=1)
y=df['label']

In [9]:
!pip install tensorflow
import tensorflow as tf

In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [11]:
### Vocabulary size
voc_size=5000

# Onehot Representation

In [12]:
messages=X.copy()

In [13]:
messages['title'][1]

"China says Trump call with Taiwan president won't change island's status"

In [14]:
messages

,id,title,text
0,0,Palestinians switch off Christmas lights in Be...,"RAMALLAH, West Bank (Reuters) - Palestinians s..."
1,1,China says Trump call with Taiwan president wo...,BEIJING (Reuters) - U.S. President-elect Donal...
2,2,FAIL! The Trump Organization’s Credit Score W...,While the controversy over Trump s personal ta...
3,3,Zimbabwe military chief's China trip was norma...,BEIJING (Reuters) - A trip to Beijing last wee...
4,4,THE MOST UNCOURAGEOUS PRESIDENT EVER Receives ...,There has never been a more UNCOURAGEOUS perso...
...,...,...,...
24348,24348,Mexico Senate committee OK's air transport dea...,MEXICO CITY (Reuters) - A key committee in Mex...
24349,24349,BREAKING: HILLARY CLINTON’S STATE DEPARTMENT G...,IF SHE S NOT TOAST NOW THEN WE RE IN BIGGER TR...
24350,24350,trump breaks from stump speech to admire beaut...,kremlin nato was created for agression \nruss...
24351,24351,NFL PLAYER Delivers Courageous Message: Stop B...,Dallas Cowboys star wide receiver Dez Bryant t...


In [15]:
import nltk
import re
from nltk.corpus import stopwords

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:

!pip install gensim

In [18]:
#dataset preprocessing
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer=WordNetLemmatizer() #lemmatizing
import gensim
from gensim.utils import simple_preprocess
corpus=[]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
for i in range(len(messages)):
  review=re.sub('[^a-zA-Z0-9]'," ",messages['title'][i])
  review=review.lower()
  review=review.split()
  review=[lemmatizer.lemmatize(word)for word in review if word not in set(stopwords.words("english"))]
  review=" ".join(review)
  corpus.append(review)


In [20]:
corpus

['palestinian switch christmas light bethlehem anti trump protest',
 'china say trump call taiwan president change island status',
 'fail trump organization credit score make laugh',
 'zimbabwe military chief china trip normal visit beijing say',
 'uncourageous president ever receives courage award proceeds whine current president',
 'suspected boko haram suicide bomber kill least 13 nigeria official',
 'watch john oliver present gop debate clowntown f ck world sh tshow 2016',
 'senate democrat ask trump attorney general pick recuse russia probe',
 'trump humiliates republican latest hissy fit siding democrat debt ceiling',
 'macy get boot loyal customer firing trump',
 'north korea nuclear test pacific logical terrifying',
 'czech police ask parliament allow prosecution prospective pm babis',
 'one democrat refuse cast electoral vote crooked hillary could end video',
 'watch senator al franken ripped ted cruz new one insulting senate hearing',
 'nearly half american still oppose repub

In [21]:
corpus[0]

'palestinian switch christmas light bethlehem anti trump protest'

In [22]:
onehot_repe=[one_hot(word,voc_size) for word in corpus]
onehot_repe

[[3960, 3219, 3697, 2701, 2851, 3264, 2893, 935],
 [2685, 2533, 2893, 1619, 4013, 1218, 2094, 3581, 1423],
 [3132, 2893, 219, 2089, 2416, 2235, 2102],
 [450, 688, 3598, 2685, 1654, 90, 1385, 3858, 2533],
 [1401, 1218, 4659, 1483, 1810, 1006, 3371, 3473, 2606, 1218],
 [1001, 1713, 49, 3386, 559, 4955, 1940, 3811, 2822, 2805],
 [3150, 1666, 677, 559, 2142, 3613, 316, 1565, 1445, 3181, 3494, 1891, 307],
 [1923, 2111, 811, 2893, 484, 477, 1052, 4213, 121, 2094],
 [2893, 3003, 3632, 4730, 2688, 516, 4098, 2111, 3618, 1418],
 [2929, 4762, 2208, 1594, 3181, 4005, 2893],
 [1415, 2039, 4124, 485, 3316, 3150, 3771],
 [3413, 1904, 811, 4629, 3540, 2872, 4009, 2256, 3422],
 [1115, 2111, 1089, 3375, 3507, 1100, 768, 1768, 81, 552, 3688],
 [3150, 2474, 79, 4274, 3158, 3902, 2569, 3479, 1115, 1662, 1923, 1483],
 [4640, 3873, 1933, 3826, 2987, 3632, 3870, 1269, 4197, 4601, 784],
 [3442, 1806, 1569, 2533, 2893, 2595, 4147, 4639, 4140],
 [135, 2210, 3160, 3471, 4035, 2210, 83, 3063, 4925],
 [133, 840, 3

In [23]:
corpus[1]

'china say trump call taiwan president change island status'

In [24]:
onehot_repe[1]

[2685, 2533, 2893, 1619, 4013, 1218, 2094, 3581, 1423]

# Embedding Representation

In [25]:
sent_size=30
embedded_docs=pad_sequences(onehot_repe,padding='pre',maxlen=sent_size)
print(embedded_docs)

[[   0    0    0 ... 3264 2893  935]
 [   0    0    0 ... 2094 3581 1423]
 [   0    0    0 ... 2416 2235 2102]
 ...
 [   0    0    0 ... 4518 2893 3869]
 [   0    0    0 ... 1161  129 3922]
 [   0    0    0 ... 2288 3672 4740]]


In [26]:
len(embedded_docs)

24353

In [27]:
#embedded_docs[0] -> corpus[0]
corpus[0]

'palestinian switch christmas light bethlehem anti trump protest'

In [28]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       3960, 3219, 3697, 2701, 2851, 3264, 2893,  935], dtype=int32)

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

embedding_vector_features = 40

model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_size))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape=(None, sent_size))  # <-- Add this line if needed
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 30, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100)            │        56,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
len(embedded_docs),y.shape

(24353, (24353,))

In [31]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [33]:
X_final

array([[   0,    0,    0, ..., 3264, 2893,  935],
       [   0,    0,    0, ..., 2094, 3581, 1423],
       [   0,    0,    0, ..., 2416, 2235, 2102],
       ...,
       [   0,    0,    0, ..., 4518, 2893, 3869],
       [   0,    0,    0, ..., 1161,  129, 3922],
       [   0,    0,    0, ..., 2288, 3672, 4740]], dtype=int32)

In [34]:
y_final

array([1, 1, 0, ..., 0, 0, 0])

In [35]:
X_final.shape,y_final.shape

((24353, 30), (24353,))

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=42)

In [47]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
267/267 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - accuracy: 0.9850 - loss: 0.0415 - val_accuracy: 0.8965 - val_loss: 0.4587
Epoch 2/10
267/267 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9858 - loss: 0.0375 - val_accuracy: 0.8942 - val_loss: 0.4659
Epoch 3/10
267/267 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - accuracy: 0.9866 - loss: 0.0351 - val_accuracy: 0.8962 - val_loss: 0.4989
Epoch 4/10
267/267 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - accuracy: 0.9893 - loss: 0.0324 - val_accuracy: 0.8919 - val_loss: 0.5058
Epoch 5/10
267/267 ━━━━━━━━━━━━━━━━━━━━ 9s 33ms/step - accuracy: 0.9902 - loss: 0.0280 - val_accuracy: 0.8928 - val_loss: 0.5497
Epoch 6/10
267/267 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - accuracy: 0.9894 - loss: 0.0269 - val_accuracy: 0.8941 - val_loss: 0.5752
Epoch 7/10
267/267 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - accuracy: 0.9932 - loss: 0.0210 - val_accuracy: 0.8863 - val_loss: 0.6213
Epoch 8/10
267/267 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - accuracy: 0.9906 - loss: 0.0233 - val_acc

# Adding dropout

In [48]:
# from tensorflow.keras.layers import Dropout
# ## Creating model
# embedding_vector_features=40
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_size))
# model.add(Dropout(0.3))
# model.add(LSTM(100))
# model.add(Dropout(0.3))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Performance Metrics And Accuracy

In [49]:
y_pred=model.predict(X_test)

229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [50]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [51]:
from sklearn.metrics import confusion_matrix

In [52]:
confusion_matrix(y_test,y_pred)

array([[2919,  363],
       [ 440, 3584]])

In [53]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8900903367095538

In [54]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.89      0.88      3282
           1       0.91      0.89      0.90      4024

    accuracy                           0.89      7306
   macro avg       0.89      0.89      0.89      7306
weighted avg       0.89      0.89      0.89      7306

